In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import time
import matplotlib.pyplot as plt
import scipy.integrate as spi
import sympy as sp
from scipy.integrate import odeint
from scipy.optimize import fsolve

In [ ]:
## define parameters

gK = 36
gNa = 120
gL = 0.3
EK = -77
ENa = 50
EL = -54.4
C = 1

I = 0
#I = 6
#I = 25

y0 = [-30,0.3]

T = 50
t0 = 0
dt = 0.001
tt = np.arange(t0, T, dt)

In [ ]:
## define functions

def HH_2D(y, t):

    #parameters
    v, n = y
    alpha_n = 0.01*(55 + v)/(1- np.exp((-55 - v)/10))
    beta_n = 0.0555*np.exp(-v/80)
    n_inf = alpha_n / (alpha_n + beta_n)
    tau_n = 1 / (alpha_n + beta_n)
    alpha_m = 0.1*(40 + v)/(1 - np.exp((-40 - v)/10))
    beta_m = 0.108*np.exp(-v/18)
    m_inf = alpha_m / (alpha_m + beta_m)

    #model
    dv = (I - (gNa*(m_inf**3))*(0.8 - n)*(v - ENa) - (gK*(n**4)*(v - EK)) - gL*(v - EL))/C
    dn = (n_inf - n) / tau_n
    return np.array([dv, dn])


def V_nullcline(v, n, I, gNa, ENa, gK, EK, gL, EL):
    alpha_m = 0.1*(40 + v)/(1 - np.exp((-40 - v)/10))
    beta_m = 0.108*np.exp(-v/18)
    m = alpha_m / (alpha_m + beta_m)
    return (I - gNa * m**3 * (0.8 - n) * (v - ENa) - gK * n**4 * (v - EK) - gL * (v - EL))

def n_nullcline(v):
    alpha_n = 0.01*(55 + v)/(1- np.exp((-55 - v)/10))
    beta_n = 0.0555*np.exp(-v/80)
    n = alpha_n / (alpha_n + beta_n)
    return n

# Generate V-n plane
v_values = np.linspace(-100, 100 ,500)
n_values = np.linspace(-0.1, 1, 500)
V, N = np.meshgrid(v_values, n_values)

# Calculate nullclines
V_nc = V_nullcline(V, N, I, gNa, ENa, gK, EK, gL, EL)
n_nc = n_nullcline(V)

In [ ]:
## solve the system

Y = spi.odeint(HH_2D, y0, tt)

plt.plot(tt, Y[:,0],'r',label='V')
plt.plot(tt, Y[:,1],'b',label='n')
plt.xlabel('Time, ms')
plt.title('Dynamics of reduced HH model for I = '+ str(I))
plt.legend()


In [ ]:
## find an equlibrium point

def HH_2D_equil(Y):

    #parameters
    v, n = Y
    alpha_n = 0.01*(55 + v)/(1- np.exp((-55 - v)/10))
    beta_n = 0.0555*np.exp(-v/80)
    n_inf = alpha_n / (alpha_n + beta_n)
    tau_n = 1 / (alpha_n + beta_n)
    alpha_m = 0.1*(40 + v)/(1 - np.exp((-40 - v)/10))
    beta_m = 0.108*np.exp(-v/18)
    m_inf = alpha_m / (alpha_m + beta_m)

    #model
    dv = (I - (gNa*(m_inf**3))*(0.8 - n)*(v - ENa) - (gK*(n**4)*(v - EK)) - gL*(v - EL))/C
    dn = (n_inf - n) / tau_n
    return [dv, dn]

v_eq, n_eq = fsolve(HH_2D_equil, y0)
print(v_eq, n_eq)

In [ ]:
## plot the phase portrait, nullclines and equilibrium point


y1 = np.linspace(-100, 100, 30)
y2 = np.linspace(0, 1.0, 30)
Y1, Y2 = np.meshgrid(y1, y2)

t = 0
u, r = np.zeros(Y1.shape), np.zeros(Y2.shape)
NI, NJ = Y1.shape

for i in range(NI):
    for j in range(NJ):
        x = Y1[i, j]
        y = Y2[i, j]
        yprime = HH_2D_2([x, y], t)
        u[i,j] = yprime[0]
        r[i,j] = yprime[1]

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(v_eq, n_eq, color='#66FF00', marker='*', s=500)
Q = plt.quiver(Y1, Y2, u, r, color='#43BFC7', width=0.002)
p = ax.plot(Y[:,0], Y[:,1],'k',label='Phase Portrait')
c1 = ax.contour(V, N, V_nc, levels=[0], colors='blue')
c2 = ax.contour(V, N, n_nc - N, levels=[0], colors='red')
ax.clabel(c1, fmt='V nullcline', inline=True, fontsize=10, manual=[(50, 0.2)])
ax.clabel(c2, fmt='n nullcline', inline=True, fontsize=10, manual=[(-75, 0.1)])
ax.set_xlabel('Voltage (V)')
ax.set_ylabel('n')
ax.set_title('Nullclines of the System, I = '+ str(I))
ax.legend()
ax.grid(True)
plt.show()

In [ ]:
##  find Jacobian, Determinant and Trace

v, n, I, gNa, ENa, gK, EK, gL, EL, C = sp.symbols('v, n, I, gNa, ENa, gK, EK, gL, EL, C')

def HH_2D_sp(Y, t):
    v, n = Y
    alpha_n = 0.01*(55 + v)/(1- sp.exp((-55 - v)/10))
    beta_n = 0.0555*sp.exp(-v/80)
    n_inf = alpha_n / (alpha_n + beta_n)
    tau_n = 1 / (alpha_n + beta_n)
    alpha_m = 0.1*(40 + v)/(1 - sp.exp((-40 - v)/10))
    beta_m = 0.108*sp.exp(-v/18)
    m_inf = alpha_m / (alpha_m + beta_m)
    dv = (I - (gNa*(m_inf**3))*(0.8 - n)*(v - ENa) - (gK*(n**4)*(v - EK)) - gL*(v - EL))/C
    dn = (n_inf - n) / tau_n
    return dv, dn



t = np.arange(0, 400, 0.1)
M = sp.Matrix(HH_2D_sp([v,n], t))
Y = sp.Matrix([v, n])
J = M.jacobian(Y)
D = J.det()
print(D)

T = J.trace()

val_num = { v:v_eq, n:n_eq, I:25, gNa:120, ENa:50, gK:36, EK:-77, gL:0.3, EL:-54.4, C:1}

Determinant_at_equilibrium = D.subs(val_num)
Trace_at_equilibrium = T.subs(val_num)
print('det:',Determinant_at_equilibrium)
print('tr:',Trace_at_equilibrium)